In [232]:
# %load helper_functions2.py

def user_event_count():
    df['user_event_count'] = df.groupby('user_created').count().max(axis=1)
    return df


def previous_payouts(df):
    new_df=pd.DataFrame(df.previous_payouts[0])
    for i in range(1,df.previous_payouts.shape[0]):
        df_element=pd.DataFrame(df.previous_payouts[i])
        new_df=pd.concat([new_df, df_element])
    return new_df

def user_df():
    unique_cols = []
    for col in columns:
        if unique_col_indicator(df,col):
            unique_cols.append(col)
    user_unique_events = df.groupby('user_created').count().max(axis=1)
    user_unique_events = pd.DataFrame(user_unique_events,columns=['user_event_count'])
    return df[unique_cols].merge(user_unique_events,how='inner',on='user_created')

def unique_col_indicator(df,col):
    try:
        col_count = df.groupby(['user_created',col])['channels'].count().shape[0]
        user_count = df.groupby(['user_created'])['channels'].count().shape[0]
        if col_count == user_count:
            return True
        else:

            return False
    except:
        pass

    
#check
def user_unique_events_col():
    user_unique_events = df.groupby('user_created').count().max(axis=1)
    user_unique_events =  pd.DataFrame(user_unique_events,columns=['user_event_count'])
    return df.merge(user_unique_events,how='inner',on='user_created')

#check
def create_target():
    df['fraud'] = df['acct_type'].map({'fraudster_event': 1,
                                   'premium': 0,
                                   'spammer_warn': 0,
                                   'fraudster': 1,
                                   'spammer_limited': 0,
                                   'spammer_noinvite': 0,
                                   'locked': 1,
                                   'tos_lock': 0,
                                   'tos_warn': 0,
                                   'fraudster_att': 1,
                                   'spammer_web': 0,
                                   'spammer': 0})
    return df

from datetime import datetime

#check
def convert_date():
    df["approx_payout_date"] = pd.to_datetime(df["approx_payout_date"], unit = 's')
    df["event_created"] = pd.to_datetime(df["event_created"], unit = 's')
    df["event_end"] = pd.to_datetime(df["event_end"], unit = 's')
    df["event_start"] = pd.to_datetime(df["event_start"], unit = 's')
    df["event_published"] = pd.to_datetime(df["event_published"], unit = 's')
    return df

def low_cor_cols(r_score):
    df[num_cols()].corr()[['fraud']].values
    corr_df = df[num_cols()].corr()[['fraud']].sort_values('fraud')
    low_corr_mask = corr_df.sort_values('fraud').abs().lt(r_score).values.reshape(1,-1)[0]
    return corr_df[low_corr_mask]

def get_corrs():
    corr_df = df[num_cols()].corr()[['fraud']].abs().sort_values('fraud')
    return corr_df

def num_cols():
    return df.describe().columns

def drop_cols():
    df1 = df.drop(low_cor_cols(.09).index.tolist(),axis=1)
    df2 = df1.drop(['acct_type','sale_duration','fraud','event_start','approx_payout_date',
                    'event_published','event_end','venue_state',
                    'ticket_types','venue_name','venue_state','venue_country','venue_address',
                    'org_desc','org_name','previous_payouts','email_domain','payee_name','name','currency','country','event_created'],axis=1)
    return df2

#check
def description_cols():
    import clean_desc
    df['description'] = df['description'].apply(strip_tags)
    clean_desc(df)
    return df

#check
def ticket_types(df):
    new_df=pd.DataFrame(df.ticket_types[0])
    result_df=new_df[['quantity_sold', 'quantity_total', 'event_id']].groupby('event_id').sum()
    result_df=result_df.join(new_df[['availability', 'cost', 'event_id']].groupby('event_id').mean())

    for i in range(1,df.shape[0]):
        try:
            df_element=pd.DataFrame(df.ticket_types[i])
            grp_ele_df=df_element[['quantity_sold', 'quantity_total', 'event_id']].groupby('event_id').sum()
            grp_ele_df=grp_ele_df.join(df_element[['availability', 'cost', 'event_id']].groupby('event_id').mean())
            result_df=pd.concat([result_df, grp_ele_df])

        except:
            continue
    result_df.reset_index(inplace=True)
    result_df.columns=['object_id', 'quantity_sold', 'quantity_total', 'availability', 'cost']
    final_df=df.set_index('object_id').join(result_df.set_index('object_id'))
    final_df.reset_index(inplace=True)
    final_df[['quantity_sold', 'quantity_total','availability', 'cost']]=final_df[['quantity_sold', 'quantity_total','availability', 'cost']].fillna(0)
    return final_df

def timedelta(field):
    return field.days

def event_times():
    df['publish_time_to_start'] =  df['event_published'] - df['event_created'] 
    df['publish_time_to_start'] = df['publish_time_to_start'].apply(timedelta)
    return df

def has_venue_data():
    df['has_state'] = (df['venue_state'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_country'] = (df['venue_country'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_address'] = (df['venue_address'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_venue_name'] = (df['venue_name'].fillna('None').replace('','None') == 'None').astype(int)
    
    df['has_org_desc'] = (df['org_desc'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_org_name'] = (df['org_name'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_venue_name'] = (df['venue_name'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_venue_name'] = (df['venue_name'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_payee_name'] = (df['payee_name'].fillna('None').replace('','None') == 'None').astype(int)
    df['org_facebook'] = df['org_facebook'].fillna(df['org_facebook'].mode()[0])
    df['org_twitter'] = df['org_twitter'].fillna(df['org_twitter'].mode()[0])
    return df

def payout_type():
    df1 = pd.get_dummies(df,columns = ['payout_type','listed'])
    return df1

In [225]:
df['org_twitter'].mode()[0]

0.0

In [306]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

import seaborn as sns
import nltk
nltk.download('words')
pd.set_option('max_colwidth', 40)
import os
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
sws = set(stopwords.words('english'))
punctuation = set(string.punctuation)
from  helper_functions import user_event_count,\
                                ticket_types,\
                                previous_payouts,\
                                user_df,\
                                unique_col_indicator,\
                                create_target,\
                                convert_date,\
                                low_cor_cols,\
                                drop_cols,\
                                description_cols,\
                                ticket_types


[nltk_data] Downloading package words to /home/ubuntu/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [216]:
df = pd.read_json('data/data.json').drop('description',axis=1)
df = ticket_types(df)

In [227]:
df = convert_date()
df = create_target()
df = user_unique_events_col()
df = event_times()
df = has_venue_data()
df = payout_type()
df = drop_cols()
df = user_unique_events_col()

In [341]:
df = df.drop('delivery_method',axis=1)
df = user_unique_events_col()

In [342]:
df.to_pickle('data/df_final.pickle')

,body_length,channels,fb_published,has_logo,name_length,org_facebook,org_twitter,sale_duration2,user_age,user_created,user_type,availability,has_state,has_country,has_address,has_venue_name,has_org_desc,has_org_name,has_payee_name,payout_type_,payout_type_CHECK,user_event_count
0,3852,5,0,0,60,0.0,0.0,33,36,1259613950,1,1.0,0,0,0,0,1,0,1,1,0,25
1,3561,8,0,1,90,0.0,0.0,16,758,1259613950,1,1.0,1,0,0,1,1,1,1,0,0,25
2,533,8,0,1,88,17.0,14.0,1,675,1259613950,1,1.0,0,0,0,0,0,0,1,1,0,25
3,630,8,0,1,67,0.0,0.0,9,889,1259613950,1,1.0,0,0,0,0,1,0,1,1,0,25
4,990,5,0,1,89,0.0,0.0,26,883,1259613950,1,1.0,0,0,0,0,1,0,1,0,0,25


In [276]:
df2 = pd.read_json('data/data.json').drop(['description'],axis=1)

In [277]:
df2['fraud'] = df2['acct_type'].map({'fraudster_event': 1,
                                   'premium': 0,
                                   'spammer_warn': 0,
                                   'fraudster': 1,
                                   'spammer_limited': 0,
                                   'spammer_noinvite': 0,
                                   'locked': 1,
                                   'tos_lock': 0,
                                   'tos_warn': 0,
                                   'fraudster_att': 1,
                                   'spammer_web': 0,
                                   'spammer': 0})

In [340]:
user_unique_events_col()

,body_length,channels,fb_published,has_logo,name_length,org_facebook,org_twitter,sale_duration2,user_age,user_created,user_type,availability,has_state,has_country,has_address,has_venue_name,has_org_desc,has_org_name,has_payee_name,payout_type_,payout_type_CHECK,user_event_count
0,3852,5,0,0,60,0.0,0.0,33,36,1259613950,1,1.0,0,0,0,0,1,0,1,1,0,25
1,3561,8,0,1,90,0.0,0.0,16,758,1259613950,1,1.0,1,0,0,1,1,1,1,0,0,25
2,533,8,0,1,88,17.0,14.0,1,675,1259613950,1,1.0,0,0,0,0,0,0,1,1,0,25
3,630,8,0,1,67,0.0,0.0,9,889,1259613950,1,1.0,0,0,0,0,1,0,1,1,0,25
4,990,5,0,1,89,0.0,0.0,26,883,1259613950,1,1.0,0,0,0,0,1,0,1,0,0,25
5,1478,8,0,1,59,17.0,14.0,10,732,1259613950,1,1.0,0,0,0,0,0,0,1,1,0,25
6,8836,6,0,1,188,0.0,0.0,3,879,1259613950,1,1.0,1,1,0,1,0,0,1,1,0,25
7,649,8,0,1,171,0.0,0.0,17,961,1259613950,1,1.0,1,1,0,0,0,0,1,1,0,25
8,593,8,0,1,165,17.0,14.0,11,750,1259613950,1,1.0,0,0,0,0,0,0,1,1,0,25
9,45882,8,1,1,72,0.0,0.0,66,0,1259613950,1,1.0,1,0,0,0,1,0,1,1,0,25


In [313]:

y = df2['fraud']
X = df
ss= StandardScaler()
X = ss.fit_transform(X)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [314]:
x_train,x_test,y_train,y_test = train_test_split(X,y)

In [315]:
lr = LogisticRegression(random_state=0,solver='sag')
lr.fit(x_train,y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [318]:
y_pred = lr.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95      3585
           1       0.00      0.00      0.00         0

   micro avg       0.91      0.91      0.91      3585
   macro avg       0.50      0.45      0.48      3585
weighted avg       1.00      0.91      0.95      3585



/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [319]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
y_pred.sum()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10

In [320]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95      3575
           1       0.01      0.40      0.02        10

   micro avg       0.91      0.91      0.91      3585
   macro avg       0.51      0.65      0.49      3585
weighted avg       1.00      0.91      0.95      3585



In [323]:
topic_vector = pd.read_pickle('data/description_topics_df.pickle')
X = topic_vector
x_train,x_test,y_train,y_test = train_test_split(X,y)

In [324]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [326]:
y_pred = rf.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      3533
           1       0.15      0.96      0.25        52

   micro avg       0.92      0.92      0.92      3585
   macro avg       0.57      0.94      0.61      3585
weighted avg       0.99      0.92      0.95      3585



In [336]:
y_test[y_pred==1].mean()

0.9615384615384616